In [1]:
import pandas as pd
import json


with open("../out/data.json") as fout:
    raw_data = json.load(fout)

user_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

{}

"""

system_template = """### Response:

{} 

"""

end_template = """{}### End"""

agg_conversations = []
for idx_thread, thread in enumerate(raw_data):
    conversations = ""
    prev_owner = ""
    
    for idx_post in range(len(thread)):
        post_curr = thread[idx_post]
        if post_curr["role"] == "user":
            if prev_owner == "user":
                conversations += "\n" + post_curr["text"]
            elif prev_owner == "system":
                conversations += "\n" + user_template.format(post_curr["text"])
            else:
                conversations += "\n" + user_template.format(post_curr["text"])

        if post_curr["role"] == "system":
            if prev_owner == "system":
                conversations += "\n" + post_curr["text"]
            elif prev_owner == "user":
                conversations += "\n" + system_template.format(post_curr["text"])
            else:
                conversations += "\n" + user_template.format(post_curr["text"])

        prev_owner = post_curr["role"]
    agg_conversations.append(end_template.format(conversations))

# create dataframe
test_conv_dataframe = pd.DataFrame(agg_conversations, columns=["conversations"])
test_conv_dataframe

,conversations
0,\nBelow is an instruction that describes a tas...
1,\nBelow is an instruction that describes a tas...
2,\nBelow is an instruction that describes a tas...
3,\nBelow is an instruction that describes a tas...
4,\nBelow is an instruction that describes a tas...
...,...
1406,\nBelow is an instruction that describes a tas...
1407,\nBelow is an instruction that describes a tas...
1408,\nBelow is an instruction that describes a tas...
1409,\nBelow is an instruction that describes a tas...


In [2]:
test_conv_dataframe.to_csv("../data/test-conversations.csv", sep="\t", index=None)

In [3]:
import re
import sys
import torch
from transformers import (LlamaTokenizer, LlamaForCausalLM)

model_path = 'openlm-research/open_llama_3b_v2'

'''model = LlamaForCausalLM.from_pretrained(
    model_path, 
    load_in_8bit=True,
    device_map='auto', 
    # If passing a string for `device_map`, please choose 'auto', 'balanced', 'balanced_low_0' or 'sequential'.
)

model.config.use_cache = False
model.config.pretraining_tp = 1'''

tokenizer = LlamaTokenizer.from_pretrained(model_path)

/scratch/users/anup/miniconda3/envs/finetune-dnabert2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-11-24 08:58:46,792] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [11]:
import re

def extract_ins_res_pairs(input_string):
    pairs_ins_res = []
    size_pairs = []
    pattern = "Below is an instruction that describes a task. Write a response that appropriately completes the request"
    matches = [m.start() for m in re.finditer(pattern, input_string)]
    #print(matches, len(input_string))
    for i in range(len(matches)-1):
        f,n = matches[i], matches[i+1]
        if i+1 < len(matches)-1:
            extracted_conv = input_string[f:n]
        else:
            extracted_conv = input_string[matches[i+1]:len(input_string)]
        input_ids = tokenizer.encode(extracted_conv, return_tensors="pt")
        pairs_ins_res.append(extracted_conv)
        size_pairs.append(len(input_ids[0]))
    return pairs_ins_res, size_pairs

ins_res_dataset = []
prs_size_dataset = []
for idx, item in test_conv_dataframe.iterrows():
    prs, size_prs = extract_ins_res_pairs(item["conversations"])
    ins_res_dataset.extend(prs)
    prs_size_dataset.extend(size_prs)
    print(idx, len(ins_res_dataset), len(prs_size_dataset))
    if idx == 2:
        break
individual_conversations = pd.DataFrame(zip(ins_res_dataset, prs_size_dataset), columns=["conversations", "tensor_size"])

0 7 7
1 15 15
2 15 15


In [13]:
size = individual_conversations["tensor_size"]

In [14]:
import numpy as np

len(size), np.mean(size), np.median(size)

(15, 659.8, 645.0)

In [15]:
small_df = individual_conversations #individual_conversations[individual_conversations["tensor_size"] <= 200]
small_df

,conversations,tensor_size
0,Below is an instruction that describes a task....,1368
1,Below is an instruction that describes a task....,497
2,Below is an instruction that describes a task....,917
3,Below is an instruction that describes a task....,732
4,Below is an instruction that describes a task....,639
5,Below is an instruction that describes a task....,103
6,Below is an instruction that describes a task....,1027
7,Below is an instruction that describes a task....,498
8,Below is an instruction that describes a task....,908
9,Below is an instruction that describes a task....,868


In [16]:
small_df.to_csv("../data/small-conversations.csv", sep="\t", index=None)